In [1]:
import numpy as np
from math import exp, log, sqrt
import matplotlib.pyplot as plt

a = 1.0
b = 2.0
N = 10
n = 5
delta = (b - a) / N
alpha = 1.7

points = [a + i * delta for i in range(N + 1)]
c = np.zeros((n + 1, 1))

def l2scalar(f, g):
    values = [f(point) * g(point) for point in points]
    return np.sum(np.array(values, dtype=np.double))

def l2norm(f):
    return l2scalar(f, f)

def f(x):
    return alpha * exp(-x) + (1 - alpha) * log(x)

def phi(i):
    if i == n + 1:
        return f
    return lambda x: x ** i

def getPhi(c):
    phi = "ф(x) = "
    for i in range(len(c)):
        if c[i] < 0:
            phi += "- " + str(abs(c[i]))
        else:
            if i != 0:
                phi += "+ "
            phi += str(c[i])
        if i != 0:
            phi += "x"
            if i != 1:
                phi += "^" + str(i)
        phi += " "
    return phi

def solution(x):
    global c
    value = 0.0
    for i in range(len(c)):
        value += c[i] * phi(i)(x)
    return value

def plotDifference(samples):
    space = np.linspace(a, b, samples)
    plt.plot(space, np.zeros(np.shape(space)))
    plt.plot(space, np.array([solution(x) - f(x) for x in space], dtype=np.double))
    plt.savefig("../TeX/Interpolation/lsqDiff.png")
    plt.show()
    
if __name__ == '__main__':
    A = np.array([[l2scalar(phi(i), phi(j)) for j in range(len(c))] for i in range(len(c))],
                 dtype = np.double)
    B = np.array([l2scalar(f, phi(i)) for i in range(len(c))], dtype = np.double)
    c = np.linalg.solve(A, B)

    print("c:\n" + str(c))
    print()

    check = [points[0] + delta / 2.6, 
             points[5] + delta / 2.6,
             points[9] + delta / 2.6]
    
    detGN1 = np.linalg.det(A)
    print("Conditioning number: nu = " + str(np.linalg.norm(A, ord=np.inf) * np.linalg.norm(np.linalg.inv(A), ord=np.inf)))
    print()
    A = np.concatenate((A, np.array([B]).T), axis=1)
    A = np.vstack((A, np.array([l2scalar(f, phi(i)) for i in range(len(c) + 1)], dtype=np.double)))
    detGN2 = np.linalg.det(A)
    
    
    [print("rn({0}) = {1}".format(x, solution(x) - f(x))) for x in check]
    print()
    
    print("Expected deficiency: " + 
          str(sqrt(abs(detGN2 / detGN1))))
    
    space = np.linspace(a, b, 1000)
    print("Real deficiency on whole interval: " + 
          str(np.max(np.abs(np.array([(solution(x) - f(x)) for x in space], dtype=np.double)))))
    print()
    
    print("Real deficiency on control points: " + 
          str(np.max(np.abs(np.array([(solution(x) - f(x)) for x in check], dtype=np.double)))))
    
    plotDifference(1000)

c:
[ 3.04926117 -4.13112546  2.50490103 -1.01148796  0.23830349 -0.02445942]

Conditioning number: nu = 624153384103.3069

rn(1.0384615384615385) = 9.374749135870886e-06
rn(1.5384615384615385) = 4.441449420605759e-06
rn(1.9384615384615385) = 7.60907463800975e-06

Expected deficiency: 1.4694497815835233e-05
Real deficiency on whole interval: 1.0453241777508282e-05

Real deficiency on control points: 9.374749135870886e-06


<Figure size 640x480 with 1 Axes>